# Bibliotecas Necessárias

In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from os.path import join, abspath

# Importando os Dados Usados

Para este caso, vamos usados o dataset de treino [Titanic - Machine Learning from Disaster](https://www.kaggle.com/competitions/titanic/data?select=train.csv)

In [2]:
df = pd.read_csv(join("..","Data","Titanic","train.csv"))

# Conhecendo Nossos Dados

In [3]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


# Visualizando todas as colunas da base de dados

In [4]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

# Após uma breve análise, foi optado por desconsiderar algumas colunas, pois as mesmas aparantemente não agregariam ao nosso próposito:

- Name
- Ticket
- Cabin
- PassengerId

In [5]:
df.drop(columns=["Name","Ticket","Cabin","PassengerId"], inplace=True)
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


# Para a maquina que trabalha com números, precisamos converter nossos textos em valóres númericos atráves de artifícios lógicos.

## Para isso, iremos realizar uma pequena simplificação dos nossos dados de string para inteiros

In [6]:
df["Sex"] = df["Sex"].replace({"male": 0, "female": 1})  # Agora todos os homens serão tratados como 0 e as mulheres como 1

C:\Users\mauri\AppData\Local\Temp\ipykernel_11380\4241701657.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Sex"] = df["Sex"].replace({"male": 0, "female": 1})  # Agora todos os homens serão tratados como 0 e as mulheres como 1


In [7]:
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,S
1,1,1,1,38.0,1,0,71.2833,C
2,1,3,1,26.0,0,0,7.9250,S
3,1,1,1,35.0,1,0,53.1000,S
4,0,3,0,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,0,27.0,0,0,13.0000,S
887,1,1,1,19.0,0,0,30.0000,S
888,0,3,1,NaN,1,2,23.4500,S
889,1,1,0,26.0,0,0,30.0000,C


# Existem valores nulos na nossa base de dados?

In [8]:
df.isnull().sum() # aparentemnete existem 177 idades faltando

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

# Como temos apenas 891 registros e 177 contem valores núlos, usaremos o artifício de calular a mediana das ideias para definir um valor para tratar dados nulos (Médiana não é sensivel a outliers, logo usamos ela ao invés da média)

In [9]:
df["Age"] = df["Age"].fillna(df["Age"].median())

In [10]:
df.isnull().sum() # prontinho, agora tratamos nossos dados núlos

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    2
dtype: int64

# Agora Iremos fazer o mesmo processo trocando a categória de embarque de um caractere para um valor inteiro. Detalhe importante: Existem valores nulos, o que nos impede de converter diretamente usando a função ord. Portanto iremos alterar primeiro os valores nulos pero caractere '?' e depois transformaremos nossos caracteres em inteiros

In [11]:
df["Embarked"] = df["Embarked"].fillna('?').apply(lambda x: ord(x))

df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,83
1,1,1,1,38.0,1,0,71.2833,67
2,1,3,1,26.0,0,0,7.9250,83
3,1,1,1,35.0,1,0,53.1000,83
4,0,3,0,35.0,0,0,8.0500,83
...,...,...,...,...,...,...,...,...
886,0,2,0,27.0,0,0,13.0000,83
887,1,1,1,19.0,0,0,30.0000,83
888,0,3,1,28.0,1,2,23.4500,83
889,1,1,0,26.0,0,0,30.0000,67


# Separando os dados entre Características e Resultado (features, targets)

- features: São os dados que iremos usar para descrever nosso resultado. (Neste caso estamos testando as caracteristicas que descrevem um sobrevivente)
- targes: São os dados que queremos predizer, neste caso os sobreviventes

In [12]:
features, target = df.drop(columns=["Survived"]), df["Survived"]

In [13]:
features

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,22.0,1,0,7.2500,83
1,1,1,38.0,1,0,71.2833,67
2,3,1,26.0,0,0,7.9250,83
3,1,1,35.0,1,0,53.1000,83
4,3,0,35.0,0,0,8.0500,83
...,...,...,...,...,...,...,...
886,2,0,27.0,0,0,13.0000,83
887,1,1,19.0,0,0,30.0000,83
888,3,1,28.0,1,2,23.4500,83
889,1,0,26.0,0,0,30.0000,67


In [14]:
target

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

# Instanciando o modelo classificador, nesse caso o KNN, e aplicando K-fold, com K = 5

In [21]:
knn = KNeighborsClassifier()


scores = cross_val_score(knn, features, target, cv=5, scoring='accuracy')

scores # A função cross_val_score tem como resultado de saída uma lista com os scores de cada iteração feita com cada parte(fold) do método k-fold cross validation


array([0.63687151, 0.64606742, 0.67977528, 0.73033708, 0.73033708])

A partir dos resultados registrados (linha 5) podemos perceber como a escolha dos dados de treino e teste, mesmo de maneira aleatória, influenciam na avaliação do desempenho do modelo. Tivemos um melhor resultado de 0.73 e um pior resultado de 0.63, uma diferença considerável se tratando de avaliação de modelos.

# Calculando a Acurácia da nossa classificação

In [22]:
print(f"Accuracy: {scores.mean():.2f} +/- {scores.std():.2f}")

Accuracy: 0.68 +/- 0.04


Nossa precisão acaba por oscilar entre 68%, podendando variar a aproximadamente 72% em bons casos ou decando para 64% em piores casos